In [106]:
#  000-000
##　初回　必須　　

import numpy as np
import pandas as pd
import pickle
import io
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

#jupyterlabを使う場合、この2行はいらない
from jupyterthemes import jtplot
jtplot.style(theme="monokai")


In [107]:
#  000-001
#　ｒａｃｅ＿ｉｎｆｏに関するコード　処理必須
ar_code_list = {\
        "10":"直",\
        "11":"左",\
        "12":"左",\
        "13":"左",\
        "14":"左",\
        "15":"左",\
        "16":"左",\
        "17":"右",\
        "18":"右",\
        "19":"右",\
        "20":"右",\
        "21":"右",\
        "22":"右",\
        "23":"左",\
        "24":"右",\
        "25":"左",\
        "26":"右",\
        "27":"左",\
        "28":"右",\
        "29":"直",\
        "51":"障",\
        "52":"障",\
        "53":"障",\
        "54":"障",\
        "55":"障",\
        "56":"障",\
        "57":"障",\
        "58":"障",\
        "59":"障"\
    } 

gr_code_list = {\
        "10":"芝",\
        "11":"芝",\
        "12":"芝",\
        "13":"芝",\
        "14":"芝",\
        "15":"芝",\
        "16":"芝",\
        "17":"芝",\
        "18":"芝",\
        "19":"芝",\
        "20":"芝",\
        "21":"芝",\
        "22":"芝",\
        "23":"ダート",\
        "24":"ダート",\
        "25":"ダート",\
        "26":"ダート",\
        "27":"サ",\
        "28":"サ",\
        "29":"ダート",\
        "51":"障害",\
        "52":"障害",\
        "53":"障害",\
        "54":"障害",\
        "55":"障害",\
        "56":"障害",\
        "57":"障害",\
        "58":"障害",\
        "59":"障害"\
    } 

class_code_list = {\
        "000":"未設定",\
        "005":"１勝クラス",\
        "010":"２勝クラス",\
        "016":"３勝クラス",\
        "701":"新馬",\
        "703":"未勝利",\
        "999":"オープン"\
    } 

grade_code_list = {\
        "A":"Ｇ１",\
        "B":"Ｇ２",\
        "C":"Ｇ３",\
        "D":"その他",\
        "E":"その他",\
        "F":"その他",\
        "G":"その他",\
        "H":"その他",\
        "L":"リステッド",\
        " ":"その他"\
    } 

juryo_shubetsu_code_list = {\
        "0":"未設定",\
        "1":"ハンデ",\
        "2":"別定",\
        "3":"馬齢",\
        "4":"定量"\
    } 

babajotai_code_list = {\
        "0":"未設定",\
        "1":"良",\
        "2":"稍重",\
        "3":"重",\
        "4":"不良"\
    } 

tenko_code_list = {\
        "0":"未設定",\
        "1":"晴",\
        "2":"曇",\
        "3":"雨",\
        "4":"小雨",\
        "5":"雪",\
        "6":"小雪"\
    } 

kyoso_shubetsu_code_list = {\
        "00":"未設定",\
        "11":"サラ２歳",\
        "12":"サラ３歳",\
        "13":"サラ３歳以上",\
        "14":"サラ４歳以上",\
        "18":"サラ障害３歳以上",\
        "19":"サラ障害４歳以上",\
        "21":"アラブ２歳",\
        "22":"アラブ３歳",\
        "23":"アラブ３歳以上",\
        "24":"アラブ４歳以上"\
    } 

keibajo_code_list = {\
        "00":"未設定",\
        "01":"札幌",\
        "02":"函館",\
        "03":"福島",\
        "04":"新潟",\
        "05":"東京",\
        "06":"中山",\
        "07":"中京",\
        "08":"京都",\
        "09":"阪神",\
        "10":"小倉"\
    } 


In [108]:
#000-010
###　初回　必須　　0010-10
### アンロード
#race_info = pd.read_pickle('../data/race_info.pickle')
race_results = pd.read_pickle('../data/race_results.pickle')
#horse_info = pd.read_pickle('../data/horse_info.pickle')
#horse_peds = pd.read_pickle('../data/horse_peds.pickle')
#race_return = pd.read_pickle('../data/race_return.pickle')

In [ ]:
horse_info.head()

In [109]:
horse_results = race_results[(race_results['kaisai_nen'] == '2024') & (race_results['data_kubun'] == '7')]

In [ ]:
race_results_sel.columns

In [110]:
horse_results['日付'] = horse_results['kaisai_nen'] + '/' + horse_results['kaisai_tsukihi'].str[0:2] + '/' + horse_results['kaisai_tsukihi'].str[2:4]
horse_results.replace({'keibajo_code': keibajo_code_list},inplace=True)
horse_results['kaisai_kai'] = horse_results['kaisai_kai'].astype('int')
horse_results['kaisai_kai'] = horse_results['kaisai_kai'].astype('str')
horse_results['kaisai_nichime'] = horse_results['kaisai_nichime'].astype('int')
horse_results['kaisai_nichime'] = horse_results['kaisai_nichime'].astype('str')
horse_results['開催'] = horse_results['kaisai_kai'] + horse_results['keibajo_code'] + horse_results['kaisai_nichime']
horse_results['天気'] = ''
horse_results['R'] = horse_results['race_bango'].astype('int')
horse_results['レース名'] = ''
horse_results['映像'] = '**'
horse_results['頭数'] = ''
horse_results['枠番'] = horse_results['wakuban']
horse_results['馬番'] = horse_results['umaban']
horse_results['オッズ'] = horse_results['tansho_odds'].str[0:3] + '.' + horse_results['tansho_odds'].str[3:4]
horse_results['人気'] = horse_results['tansho_ninkijun']
horse_results['着順'] = horse_results['kakutei_chakujun']
horse_results['騎手'] = horse_results['kishu_code']
horse_results['斤量'] = horse_results['futan_juryo'].str[0:2] + '.' + horse_results['futan_juryo'].str[2:3]
horse_results['距離'] = ''
horse_results['馬場'] = ''
horse_results['馬場指数'] = '**'
horse_results['タイム'] = horse_results['soha_time'].str[0:1] + ':' + horse_results['soha_time'].str[1:3] + '.' +horse_results['soha_time'].str[3:4]

horse_results['着差符号'] = horse_results['time_sa'].str[0:1]
horse_results['着差タイム'] = horse_results['time_sa'].str[1:3] + '.' + horse_results['time_sa'].str[3:4]
horse_results['着差タイム'] = horse_results['着差タイム'].astype('float')

horse_results['着差'] = horse_results.apply(\
        lambda x: x['着差タイム'] if x['着差符号'] == '+' else 0,\
                    axis=1)

horse_results['タイム指数'] = '**'
horse_results['通過'] = ''
horse_results['ペース'] = ''
horse_results['上がり'] = horse_results['kohan_3f'].str[0:2] + '.' + horse_results['kohan_3f'].str[2:3]
#horse_results['zogen_sa'] = horse_results['zogen_sa'].astype('int')
horse_results['馬体重'] = horse_results['bataiju'] + '(' + horse_results['zogen_fugo'] + horse_results['zogen_sa'] + ')'
horse_results['厩舎ｺﾒﾝﾄ'] = ''
horse_results['備考'] = ''
horse_results['勝ち馬(2着馬)'] = ''
horse_results['賞金'] = horse_results['kakutoku_honshokin']

horse_results = horse_results[['日付',\
                        '開催',\
                        '天気',\
                        'R',\
                        'レース名',\
                        '映像',\
                        '頭数',\
                        '枠番',\
                        '馬番',\
                        'オッズ',\
                        '人気',\
                        '着順',\
                        '騎手',\
                        '斤量',\
                        '距離',\
                        '馬場',\
                        '馬場指数',\
                        'タイム',\
                        '着差',\
                        'タイム指数',\
                        '通過',\
                        'ペース',\
                        '上がり',\
                        '馬体重',\
                        '厩舎ｺﾒﾝﾄ',\
                        '備考',\
                        '勝ち馬(2着馬)',\
                        '賞金']]
horse_results = horse_results.rename_axis(None)
#horse_results.head()


C:\Users\mk198\AppData\Local\Temp\ipykernel_21320\245358812.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horse_results['日付'] = horse_results['kaisai_nen'] + '/' + horse_results['kaisai_tsukihi'].str[0:2] + '/' + horse_results['kaisai_tsukihi'].str[2:4]
C:\Users\mk198\AppData\Local\Temp\ipykernel_21320\245358812.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horse_results.replace({'keibajo_code': keibajo_code_list},inplace=True)
C:\Users\mk198\AppData\Local\Temp\ipykernel_21320\245358812.py:3: Se

In [ ]:
horse_results.head()

In [111]:
#000-010
###　初回　必須　　0010-10
### アンロード
#race_info = pd.read_pickle('../data/race_info.pickle')
#race_results = pd.read_pickle('../data/race_results.pickle')
#horse_info = pd.read_pickle('../data/horse_info.pickle')
horse_info = pd.read_pickle('../data/horse_peds.pickle')
#race_return = pd.read_pickle('../data/race_return.pickle')

In [112]:
horse_info['生年月日'] = pd.to_datetime(horse_info["seinengappi"],format='%Y%m%d')
horse_info['調教師'] = horse_info['chokyoshimei_ryakusho']
horse_info['馬主'] = horse_info['banushimei']
horse_info['生産者'] = horse_info['seisanshamei']
horse_info['産地'] = horse_info['sanchimei']
horse_info['セリ取引価格'] = ''
horse_info['獲得賞金'] = horse_info['heichi_honshokin_ruikei'].astype('int')
horse_info['獲得賞金'] = horse_info['獲得賞金'] / 100
horse_info['通算成績'] = ''
horse_info['主な勝鞍'] = ''
horse_info['近親馬'] = ''
horse_info['trainer_id'] = horse_info['chokyoshi_code']
horse_info['owner_id'] = horse_info['banushi_code']
horse_info['breeder_id'] = horse_info['seisansha_code']
horse_info['募集情報'] = ''

horse_info = horse_info.set_index('ketto_toroku_bango')
horse_info = horse_info[['生年月日',\
                        '調教師',\
                        '馬主',\
                        '生産者',\
                        '産地',\
                        'セリ取引価格',\
                        '獲得賞金',\
                        '通算成績',\
                        '主な勝鞍',\
                        '近親馬',\
                        'trainer_id',\
                        'owner_id',\
                        'breeder_id',\
                        '募集情報']]
horse_info = horse_info.rename_axis(None)


In [ ]:
horse_info.head()

In [113]:
#000-010
###　初回　必須　　0010-10
### アンロード
race_info = pd.read_pickle('../data/race_info.pickle')
#race_results = pd.read_pickle('../data/race_results.pickle')
#horse_info = pd.read_pickle('../data/horse_info.pickle')
#horse_peds = pd.read_pickle('../data/horse_peds.pickle')
#race_return = pd.read_pickle('../data/race_return.pickle')

In [ ]:
race_info['race_class'].value_counts()
#pd.set_option('display.max_columns', 100)
#race_info.head()

In [114]:
race_info['course_len'] = race_info['kyori']
race_info['around'] = race_info['track_code']
race_info.replace({'around': ar_code_list},inplace=True)
race_info.replace({'tenko_code': tenko_code_list},inplace=True)
race_info['weather'] = race_info['tenko_code']
race_info['race_type'] = race_info['track_code']
race_info.replace({'race_type': gr_code_list},inplace=True)
race_info.replace({'race_class': class_code_list},inplace=True)
race_info['ground_state'] = race_info.apply(\
        lambda x: x['babajotai_code_shiba'] if x['race_type'] == '芝' else x['babajotai_code_dirt'] ,\
                    axis=1)
race_info.replace({'ground_state': babajotai_code_list},inplace=True)
race_info['race_class'] = race_info['kyoso_joken_code']
race_info.replace({'race_class': class_code_list},inplace=True)
race_info.replace({'grade_code': grade_code_list},inplace=True)
race_info['race_class'] = race_info.apply(\
        lambda x: x['race_class'] if x['grade_code'] == 'その他' else x['grade_code'] ,\
                    axis=1)
race_info['date2'] = race_info['kaisai_nen'] + race_info['kaisai_tsukihi']
race_info['date'] = pd.to_datetime(race_info["date2"],format='%Y%m%d')
#race_info['産地'] = race_info['sanchimei']
race_info = race_info[['course_len',\
                        'around',\
                        'weather',\
                        'race_type',\
                        'ground_state',\
                        'date',\
                        'race_class']]
race_info = race_info.rename_axis(None)



In [ ]:
race_info.head()

In [115]:
horse_results = pd.merge(horse_results,race_info,left_index=True,right_index=True,how='left')

horse_results['天気'] = horse_results['weather']
horse_results['距離'] = horse_results['course_len']
horse_results['馬場'] = horse_results['ground_state']

horse_results = horse_results[['日付',\
                        '開催',\
                        '天気',\
                        'R',\
                        'レース名',\
                        '映像',\
                        '頭数',\
                        '枠番',\
                        '馬番',\
                        'オッズ',\
                        '人気',\
                        '着順',\
                        '騎手',\
                        '斤量',\
                        '距離',\
                        '馬場',\
                        '馬場指数',\
                        'タイム',\
                        '着差',\
                        'タイム指数',\
                        '通過',\
                        'ペース',\
                        '上がり',\
                        '馬体重',\
                        '厩舎ｺﾒﾝﾄ',\
                        '備考',\
                        '勝ち馬(2着馬)',\
                        '賞金']]


In [116]:
horse_results.head()

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,人気,着順,騎手,斤量,距離,馬場,馬場指数,タイム,着差,タイム指数,通過,ペース,上がり,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
202406010101,2024/01/06,1中山1,晴,1,,**,,1,01,421.5,15,16,01207,54.0,1200,良,**,1:16.8,4.2,**,,,39.6,470(+002),,,,00000000
202406010101,2024/01/06,1中山1,晴,1,,**,,1,02,007.8,04,07,01161,57.0,1200,良,**,1:13.7,1.1,**,,,39.8,528(+004),,,,00000000
202406010101,2024/01/06,1中山1,晴,1,,**,,2,03,422.9,16,14,01109,57.0,1200,良,**,1:16.2,3.6,**,,,39.0,456(-004),,,,00000000
202406010101,2024/01/06,1中山1,晴,1,,**,,2,04,003.9,02,04,01150,57.0,1200,良,**,1:13.2,0.6,**,,,38.7,420(+002),,,,00008300
202406010101,2024/01/06,1中山1,晴,1,,**,,3,05,107.1,11,13,00422,57.0,1200,良,**,1:15.3,2.7,**,,,38.7,460(-004),,,,00000000
